In [2]:
import pandas as pd
import numpy as np
import math

In [372]:
class LogisticRegression(object):
    def __init__(self):
        self.w = None
        self.X = None
        self.y = None
        
        
    def train(self,X,y,batch_size=None,epoch=1000,l_rate=0.3):
        self.X = np.hstack((np.ones((X.shape[0],1)),X))
        self.y = y
        self.w = np.zeros( (self.X.shape[1],1) )
        s_gra = np.zeros( (self.X.shape[1],1) )
        
        num_data = X.shape[0]
        
        if not batch_size:
            batch_size = self.X.shape[0]
        
        num_batch = math.ceil(num_data/batch_size)
        X = self.X
        y = self.y
        
        for i in range(epoch):
            # w,y_plum,s_gra = self.__update(l_rate,s_gra)
            
            for i in range(num_batch):
                
                X_batch = X[i*batch_size:min((i+1)*batch_size,num_data)]
                y_batch = y[i*batch_size:min((i+1)*batch_size,num_data)]
                hypo = np.dot(X_batch,self.w) 
                y_predict = self.sigmoid(hypo)
                error = y_batch - y_predict

                gra= -1*np.dot(X_batch.T,error)


                s_gra = s_gra + gra**2  
            
                self.w = self.w - l_rate/np.sqrt(s_gra)*gra
                

        
    def softmax(self,z):
        z = np.exp(z)
        total = np.sum(z)
        return z/total
    
    def cost_function(self,y_hat,y):
        cost_sum=0
        for i,val in enumerate(y_hat):
            if val[0]==1:
                cost_sum+=y[i][0]
            elif val[0]==0:
                cost_sum+=(1-y[i][0])
                
        return -1*cost_sum
                
        
    def cost_function_derivative(self,X,y_hat,y_sig):
        diff = y_hat - y_sig
        grad = -1*np.dot(X.T,diff)
        return grad
        
    
        
    def predict(self,test_X):
        test_X = np.hstack((np.ones((test_X.shape[0],1)),test_X))
        y_plum = np.dot(test_X,self.w)
        #temp_y = self.sigmoid(y_plum)
        res = [1 if i>=0 else 0 for i in y_plum]
        #res = [1 if i>=0.5 else 0 for i in temp_y]

        return res

    def sigmoid(self,z):
        res = 1/(1.0+np.exp(-z))
        return np.clip(res,0.00000000000001,0.99999999999999)

    
    @staticmethod
    def standarize(df):
        res = df
        cols=res.columns
        for col in cols:
            max_num = res[col].max()
            min_num = res[col].min()
            diff = max_num-min_num
            res[col] = (res[col]-min_num)/diff
        return res
    
    @staticmethod
    def standarize_std(df):
        res=df
        cols = res.columns
        for col in cols:
            std = np.std(res[col])
            mean = np.mean(res[col])
            res[col]=(res[col]-mean)/std
        return res
        

In [189]:
income = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

In [190]:
y_train = income["income"]==' <=50K'
y_train = np.array([0 if i else 1 for i in y_train]).reshape((-1,1))

X_train = income.drop("income",axis=1)
all_data = pd.concat([X_train,test],axis=0)

In [191]:
discrete_col = ['workclass','education','marital_status','occupation', 'relationship', 'race','native_country']
continuous_col = ['age','fnlwgt','sex','capital_gain', 'capital_loss','hours_per_week']

In [192]:
discrete = pd.get_dummies(all_data[discrete_col])
continuous = all_data[continuous_col]
data = pd.concat([continuous,discrete],axis=1)

In [193]:
sex_mapping={
    ' Male':0,
    ' Female':1
}

data["sex"]=data["sex"].map(sex_mapping)

In [194]:
#columns = ['age','fnlwgt','capital_gain','capital_loss','hours_per_week']
#data = data[columns]

In [373]:
lr = LogisticRegression()
data=lr.standarize_std(data)

In [374]:
X_train = data.iloc[:income.shape[0],:].values
X_test = data.iloc[income.shape[0]:,:].values

In [375]:
lr.train(X_train,y_train,batch_size=600,epoch=100,l_rate=0.4)
predictions=lr.predict(X_test)

In [376]:
ans=pd.read_csv("correct_answer.csv")

In [377]:
ans_list=list(ans["label"])
count=0
for i,val in enumerate(ans_list):
    if val == predictions[i]:
         count+=1
rate = count/len(ans_list)
print(rate)

0.8526503286038941


In [370]:
def softmax(z):
    z = np.exp(z)
    total = np.sum(z)
    return z/total

In [371]:
a = [[1],[2],[3]]
softmax(a)


array([[0.09003057],
       [0.24472847],
       [0.66524096]])

In [378]:
from sklearn.linear_model import LogisticRegression

In [379]:
test_lr = LogisticRegression()
test_lr.fit(X_train,y_train)

/Users/phx/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)